In [2]:
import sys
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import ConfusionMatrixDisplay
sys.path.append('..')
print(sys.path)
from nlp_pipeline.feature_extraction import FeatureExtractor
from nlp_pipeline.preprocess_text import TextPreprocessor
from model.evaluate_model import ModelEvaluator
from model.train_baseline import BaselineTrainer
from interpretation.lime_interpreter import LimeTextInterpreter
from interpretation.shap_interpreter import ShapInterpreter
from nlp_pipeline.back_translator import BackTranslationAugmentor
from nlp_pipeline.embedding_oversample import EmbeddingOversampler

['d:\\aco\\research\\Asd-classification\\notebook', 'c:\\Users\\MufliDevs\\anaconda3\\python312.zip', 'c:\\Users\\MufliDevs\\anaconda3\\DLLs', 'c:\\Users\\MufliDevs\\anaconda3\\Lib', 'c:\\Users\\MufliDevs\\anaconda3', '', 'c:\\Users\\MufliDevs\\anaconda3\\Lib\\site-packages', 'c:\\Users\\MufliDevs\\anaconda3\\Lib\\site-packages\\win32', 'c:\\Users\\MufliDevs\\anaconda3\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\MufliDevs\\anaconda3\\Lib\\site-packages\\Pythonwin', 'c:\\Users\\MufliDevs\\anaconda3\\Lib\\site-packages\\setuptools\\_vendor', '..']


Load dataset + Augmented

In [ ]:
df = df = pd.read_csv('../data/feature/merged_final.csv', encoding='latin1')
columns = ['label', 'transcription', 'total_words', 'unique_words', 'num_sentences',
                   'stopwords', 'num_adjectives', 'num_nouns', 'num_verbs', 'num_adverbs',
                   'type_token_ratio', 'avg_words_per_sentence']
df = df[columns]
augmentor = BackTranslationAugmentor()
df_augmented = augmentor.augment_dataframe(df)
print(df_augmented.head)
df_augmented.to_csv('../data/feature/1_augmented.csv', index=False)

Preprocessing text

In [3]:
df = df = pd.read_csv('../data/feature/combined_augmented_asd.csv', encoding='latin1')
text_preprocessor = TextPreprocessor()
df['clean_text'] = df['transcription'].apply(text_preprocessor.preprocess)
df[['transcription', 'clean_text']].head()

,transcription,clean_text
0,Tunjuk kaka coba,tunjuk kaka coba
1,Inii!,ini
2,Siapa namamu?,siapa nama
3,Iyaaaa?,
4,Kenalan duluu!,kenal duluu


In [9]:
df['clean_text'].size

8380

In [10]:
df['label'].value_counts()
df['label'].value_counts()

label
NON ASD    4648
ASD        3732
Name: count, dtype: int64

Ekstraksi fitur with TFIDF

In [ ]:
extractor = FeatureExtractor()
X = extractor.fit_transform_tfidf(df['clean_text'])
y = df['label'].values
print(X.shape, y.shape)

Oversampling with TFIDF

In [ ]:
X_dense = extractor.fit_transform_tfidf(df['clean_text']).toarray()

ling_cols = extractor.linguistic_cols
X_ling = df[ling_cols].values

y = (df['label'].str.upper().str.strip() == 'ASD').astype(int).values

oversampler = EmbeddingOversampler()
X_embed_bal, X_ling_bal, y_bal = oversampler.oversample(X_dense, X_ling, y)


Ekstraksi fitur with IndoBERT

In [4]:
extractor = FeatureExtractor()
y = (df['label'].str.upper().str.strip() == 'ASD').astype(int).values
X = extractor.encode_series_bert(df['clean_text'][:len(df['label'])])
unique, counts = np.unique(y, return_counts=True)
print(dict(zip(unique, counts)))

ling_cols = extractor.linguistic_cols
X_ling = df[ling_cols].values[:len(y)]

oversampler = EmbeddingOversampler()
X_embed_bal, X_ling_bal, y_bal = oversampler.oversample(X, X_ling, y)

print(X_embed_bal.shape, y_bal.shape)

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

{0: 4648, 1: 3732}
Before Oversampling: Counter({0: 4648, 1: 3732})
After Oversampling : Counter({0: 4648, 1: 4648})
(9296, 768) (9296,)


In [11]:
print(X_embed_bal.shape, y_bal.shape)

(9296, 768) (9296,)


SVC WITH 5 FOLD

In [ ]:
trainer = BaselineTrainer()

evaluator = ModelEvaluator(model_type='svm',pos_label=1)

results = evaluator.cross_validate_oversample_with_confusionmatrix(X_embed_bal,y_bal)

USE THIS IF YOU NOT USING BERT

In [ ]:
final_model = trainer.train(X_embed_bal, y_bal)

lime_interpreter = LimeTextInterpreter(
    model=final_model,
    vectorizer=extractor.get_tfidf_vectorizer(),
    class_names=['NON ASD', 'ASD']
)

asd_samples = df[df['label'] == 'ASD'].sample(n=12, random_state=42)
non_asd_samples = df[df['label'] == 'NON ASD'].sample(n=12, random_state=42)
lime_samples = pd.concat([asd_samples, non_asd_samples]).reset_index(drop=True)

lime_interpreter.save_lime_explanation_to_csv(lime_samples,"../reports/lime_results.csv")

In [ ]:
final_model = BaselineTrainer('svm').train(X_embed_bal,y_bal)
interpreter = ShapInterpreter(model=final_model)
sampled_df = df.sample(n=100, random_state=42)
texts = sampled_df['clean_text'].tolist()
labels = sampled_df['label'].tolist()

shap_values = interpreter.explain(texts)
interpreter.save_shap_explanations_to_csv(
    shap_values=shap_values,
    texts=texts,
    output_path="../reports/shap_results_SVM_oversample+bert.csv",
    true_labels=labels
)

LOGISTIC REGRETION WITH 5 FOLD

In [ ]:
trainer = BaselineTrainer()
evaluator = ModelEvaluator(pos_label=1)

results = evaluator.cross_validate_oversample_with_confusionmatrix(X_embed_bal,y_bal)

In [ ]:
final_model = trainer.train(X, y)

lime_interpreter = LimeTextInterpreter(
    model=final_model,
    vectorizer=extractor.get_tfidf_vectorizer(),
    class_names=['NON ASD', 'ASD']
)

asd_samples = df[df['label'] == 'ASD'].sample(n=12, random_state=42)
non_asd_samples = df[df['label'] == 'NON ASD'].sample(n=12, random_state=42)
lime_samples = pd.concat([asd_samples, non_asd_samples]).reset_index(drop=True)

lime_interpreter.save_lime_explanation_to_csv(lime_samples,"../reports/lime_results_LOGRES.csv")

SVM WITH 10 FOLD

In [ ]:
trainer = BaselineTrainer()
evaluator = ModelEvaluator(model_type='svm', n_splits= 10, random_state=32,pos_label=1)

results = evaluator.cross_validate_oversample_with_confusionmatrix(X_embed_bal,y_bal)


[Oversampled] Fold 1
              precision    recall  f1-score   support

           0      0.928     0.834     0.879       465
           1      0.850     0.935     0.890       465

    accuracy                          0.885       930
   macro avg      0.889     0.885     0.885       930
weighted avg      0.889     0.885     0.885       930

Confusion Matrix:
 [[388  77]
 [ 30 435]]
Confusion Matrix (Fold 1):
[[388  77]
 [ 30 435]]

[Oversampled] Fold 2
              precision    recall  f1-score   support

           0      0.933     0.804     0.864       465
           1      0.828     0.942     0.881       465

    accuracy                          0.873       930
   macro avg      0.880     0.873     0.873       930
weighted avg      0.880     0.873     0.873       930

Confusion Matrix:
 [[374  91]
 [ 27 438]]
Confusion Matrix (Fold 2):
[[374  91]
 [ 27 438]]

[Oversampled] Fold 3
              precision    recall  f1-score   support

           0      0.921     0.847     0.8

In [ ]:
final_model = trainer.train(X, y)

lime_interpreter = LimeTextInterpreter(
    model=final_model,
    vectorizer=extractor.get_tfidf_vectorizer(),
    class_names=['NON ASD', 'ASD']
)

asd_samples = df[df['label'] == 'ASD'].sample(n=12, random_state=42)
non_asd_samples = df[df['label'] == 'NON ASD'].sample(n=12, random_state=42)
lime_samples = pd.concat([asd_samples, non_asd_samples]).reset_index(drop=True)

lime_interpreter.save_lime_explanation_to_csv(lime_samples,"../reports/lime_results_SVM10FOLD.csv")

In [ ]:
final_model = BaselineTrainer('svm').train(X,y)
interpreter = ShapInterpreter(model=final_model)
sampled_df = df.sample(n=100, random_state=42)
texts = sampled_df['clean_text'].tolist()
labels = sampled_df['label'].tolist()

shap_values = interpreter.explain(texts)
interpreter.save_shap_explanations_to_csv(
    shap_values=shap_values,
    texts=texts,
    output_path="../reports/shap_results_SVM_10_fold.csv",
    true_labels=labels
)

LINEAR REGRETION WITH 10 FOLD

In [ ]:
trainer = BaselineTrainer()
evaluator = ModelEvaluator(n_splits= 10, random_state=32,pos_label=1)

results = evaluator.cross_validate_oversample_with_confusionmatrix(X_embed_bal,y_bal)

In [ ]:
final_model = BaselineTrainer('svm').train(X,y)
interpreter = ShapInterpreter(model=final_model)
sampled_df = df.sample(n=100, random_state=42)
texts = sampled_df['clean_text'].tolist()
labels = sampled_df['label'].tolist()

shap_values = interpreter.explain(texts)
interpreter.save_shap_explanations_to_csv(
    shap_values=shap_values,
    texts=texts,
    output_path="../reports/shap_results_Logres_10_fold.csv",
    true_labels=labels
)


In [ ]:
interpreter.visualize(shap_values= shap_values)

FUSION

In [ ]:
extractor = FeatureExtractor()
X = extractor.extract_fused_features_bert(df)
y = (df['label'].str.upper().str.strip() == 'ASD').astype(int).values
ling_cols = extractor.linguistic_cols
X_ling = df[ling_cols].values
oversampler = EmbeddingOversampler()
X_embed_bal, X_ling_bal, y_bal = oversampler.oversample(X, X_ling, y)

print(X_embed_bal.shape, y.shape)

Fusion with tfidf

In [ ]:
extractor = FeatureExtractor()
X = extractor.extract_fused_features_tfidf(df, fit=True)
y = (df['label'].str.upper().str.strip() == 'ASD').astype(int).values
ling_cols = extractor.linguistic_cols
X_ling = df[ling_cols].values
oversampler = EmbeddingOversampler()
X_embed_bal, X_ling_bal, y_bal = oversampler.oversample(X, X_ling, y)

print(X_embed_bal.shape, y.shape)

In [ ]:
evaluator = ModelEvaluator(model_type='svm', n_splits= 10, random_state=32)
results = evaluator.cross_validate_with_confusionmatrix(X, y)


Testing Model

In [ ]:
trainer = BaselineTrainer(model_type='svm')
model = trainer.train(X_embed_bal, y_bal)

new_text = "Apakah kamu suddah makan?"
clean_text = text_preprocessor.preprocess(new_text)
X_new = extractor.encode_series_bert([clean_text])
predicted_label = model.predict(X_new)[0]
print(f"Predicted label: {predicted_label}")

In [ ]:
new_text = "Apakah kamu sudah makan?"
clean_text = text_preprocessor.preprocess(new_text)
X_new = extractor.encode_series_bert([clean_text])
predicted_label = model.predict(X_new)[0]
print(f"Predicted label: {predicted_label}")

In [1]:
accuracy   = [0.877, 0.870, 0.891, 0.863, 0.875, 0.856, 0.867, 0.861, 0.889, 0.872]
precision  = [0.839, 0.832, 0.868, 0.838, 0.857, 0.811, 0.836, 0.836, 0.866, 0.842]
recall     = [0.933, 0.927, 0.923, 0.901, 0.901, 0.929, 0.912, 0.899, 0.920, 0.916]
f1_scores  = [0.884, 0.877, 0.895, 0.868, 0.878, 0.866, 0.872, 0.866, 0.893, 0.877]
folds = [f"Fold {i+1}" for i in range(10)]

mean_acc = sum(accuracy)/len(accuracy)
mean_prec = sum(precision)/len(precision)
mean_rec = sum(recall)/len(recall)
mean_f1 = sum(f1_scores)/len(f1_scores)

print (mean_acc, mean_f1, mean_prec, mean_rec)

0.8721 0.8775999999999999 0.8425 0.9160999999999999


Bar plot

In [ ]:
colors = ['green' if score > mean_f1 else 'skyblue' for score in f1_scores]
plt.figure(figsize=(10, 6))
bars = plt.bar(folds, f1_scores, color=colors)
plt.axhline(y=mean_f1, color='red', linestyle='--', label=f'Mean F1-score ({mean_f1:.3f})')
plt.title('F1-Score per Fold (Oversampled)')
plt.xlabel('Fold')
plt.ylabel('F1-score')
plt.ylim(0.8, 0.92)
plt.legend()
plt.tight_layout()
plt.show()

Line plot

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(folds, accuracy, label='Accuracy', marker='o')
plt.plot(folds, precision, label='Precision', marker='o')
plt.plot(folds, recall, label='Recall', marker='o')
plt.plot(folds, f1_scores, label='F1-score', marker='o')
plt.title('Cross-Validation Metrics per Fold')
plt.xlabel('Fold')
plt.ylabel('Score')
plt.ylim(0.8, 1)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

Confusion Matrix

In [ ]:
conf_matrices = [
  np.array([[382,  83], [ 31, 434]]),  # Fold 1
    np.array([[378,  87], [ 34, 431]]),  # Fold 2
    np.array([[400,  65], [ 36, 429]]),  # Fold 3
    np.array([[384,  81], [ 46, 419]]),  # Fold 4
    np.array([[395,  70], [ 46, 419]]),  # Fold 5
    np.array([[364, 101], [ 33, 432]]),  # Fold 6
    np.array([[382,  83], [ 41, 423]]),  # Fold 7
    np.array([[383,  82], [ 47, 417]]),  # Fold 8
    np.array([[398,  66], [ 37, 428]]),  # Fold 9
    np.array([[384,  80], [ 39, 426]])   # Fold 10
]
agg_conf_matrix = sum(conf_matrices)
print("Aggregated Confusion Matrix:\n", agg_conf_matrix)

In [ ]:
plt.figure(figsize=(6, 5))
sns.heatmap(agg_conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=["Predicted 0", "Predicted 1"],
            yticklabels=["Actual 0", "Actual 1"])
plt.title("Support vector machine (10-Fold Oversampled)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.tight_layout()
plt.show()